Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier

Load Dataset

In [3]:
# load dataset
df = np.loadtxt("data_oversampled.csv", skiprows = 1, delimiter=",")
# split into input (X) and output (Y) variables
x = df[:,0:4]
y = df[:,4]

Standardisation

In [4]:
#Train, Test and Split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, random_state=101)
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

Tuning the dropout regularization rate and weight constraints

In [12]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 5 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(128, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=0)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.802518 using {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.802518 (0.041033) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.802518 (0.041033) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.802518 (0.041033) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.802518 (0.041033) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.802518 (0.041033) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.787270 (0.020109) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.787270 (0.020109) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.787270 (0.020109) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.787270 (0.020109) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.787270 (0.020109) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.793365 (0.027525) with: {'model__dr

In [13]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 4 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(64, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=0)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.797931 using {'model__dropout_rate': 0.3, 'model__weight_constraint': 1.0}
0.782752 (0.036016) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.782752 (0.036016) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.782752 (0.036016) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.782752 (0.036016) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.782752 (0.036016) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.784267 (0.034684) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.784267 (0.034684) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.784267 (0.034684) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.784267 (0.034684) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.784267 (0.034684) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.790342 (0.031958) with: {'model__dr

In [7]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 3 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(32, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.797952 using {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.790356 (0.038178) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.790356 (0.038178) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.790356 (0.038178) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.790356 (0.038178) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.790356 (0.038178) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.797952 (0.037540) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.797952 (0.037540) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.797952 (0.037540) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.797952 (0.037540) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.797952 (0.037540) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.793386 (0.033893) with: {'model__dr

In [8]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 2 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(16, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.788847 using {'model__dropout_rate': 0.2, 'model__weight_constraint': 1.0}
0.775142 (0.031141) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.775142 (0.031141) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.775142 (0.031141) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.775142 (0.031141) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.775142 (0.031141) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.779715 (0.037129) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.779715 (0.037129) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.779715 (0.037129) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.779715 (0.037129) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.779715 (0.037129) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.788847 (0.044526) with: {'model__dr

In [9]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# 1 hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(8, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.764536 using {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.763021 (0.048677) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.763014 (0.045940) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.763014 (0.045940) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.763014 (0.045940) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.763014 (0.045940) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.761492 (0.043839) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.764536 (0.046576) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.764536 (0.046576) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.764536 (0.046576) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.764536 (0.046576) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.758454 (0.045167) with: {'model__dr

In [10]:
# fixate random seed to reproduce the same sequence
seed = 7
tf.random.set_seed(seed)
# NO hidden layers
# Function to create model, required for KerasClassifier
def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(4, input_shape=(4,), activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(model=create_model, epochs=150, batch_size=16, verbose=0, random_state=101)
# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

#param_grid = dict(model__dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(scaled_X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\mytze\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.759956 using {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.758434 (0.038715) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.755390 (0.035243) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.755390 (0.035243) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.755390 (0.035243) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.755390 (0.035243) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.759956 (0.040511) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.756912 (0.036957) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.756912 (0.036957) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.756912 (0.036957) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.756912 (0.036957) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.750830 (0.032781) with: {'model__dr